# Create a simple tensor with random items

In [1]:
import numpy as np

# Suppress scientific notation
np.set_printoptions(suppress=True)

# Generate randomly distributed parameters
params = np.random.uniform(low=-50, high=150, size=20)

# Make sure important values are at the beginning for better debugging
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

# Round each number to the second decimal place
params = np.round(params, 2)

# Print the parameters
print(params)

[126.34 -48.25   0.   -26.62 -25.33 125.34  59.8  124.3   84.23  83.14
  46.56  85.25 -28.93 -26.43 -47.25   1.23  11.86  -4.82  -3.51 -25.49]


# Define the quantization methods and quantize

In [4]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q

def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    # Calculate the scale and zero point
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits-1)
    zero = -1*np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantize(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale

def symmetric_dequantize(params_q: np.array, scale: float) -> np.array:
    return params_q * scale

def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    # Calculate the scale
    alpha = np.max(np.abs(params))
    scale = alpha / (2**(bits-1)-1)
    lower_bound = -2**(bits-1)
    upper_bound = 2**(bits-1)-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale

def quantization_error(params: np.array, params_q: np.array):
    # calculate the MSE
    return np.mean((params - params_q)**2)

(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original:
[126.34 -48.25   0.   -26.62 -25.33 125.34  59.8  124.3   84.23  83.14
  46.56  85.25 -28.93 -26.43 -47.25   1.23  11.86  -4.82  -3.51 -25.49]

Asymmetric scale: 0.6846666666666666, zero: 70.0
[255   0  70  31  33 253 157 252 193 191 138 195  28  31   1  72  87  63
  65  33]

Symmetric scale: 0.9948031496062992
[127 -49   0 -27 -25 126  60 125  85  84  47  86 -29 -27 -47   1  12  -5
  -4 -26]


In [5]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequantize Asymmetric:')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[126.34 -48.25   0.   -26.62 -25.33 125.34  59.8  124.3   84.23  83.14
  46.56  85.25 -28.93 -26.43 -47.25   1.23  11.86  -4.82  -3.51 -25.49]

Dequantize Asymmetric:
[126.66 -47.93   0.   -26.7  -25.33 125.29  59.57 124.61  84.21  82.84
  46.56  85.58 -28.76 -26.7  -47.24   1.37  11.64  -4.79  -3.42 -25.33]

Dequantize Symmetric:
[126.34 -48.75   0.   -26.86 -24.87 125.35  59.69 124.35  84.56  83.56
  46.76  85.55 -28.85 -26.86 -46.76   0.99  11.94  -4.97  -3.98 -25.86]


In [6]:
# Calculate the quantization error
print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymmetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(params, params_deq_symmetric), 2)}')

  Asymmetric error: 0.04
   Symmetric error: 0.09
